In [1]:
!pip install easyocr


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import matplotlib.pyplot as plt
import cv2
import easyocr
from IPython.display import Image

In [3]:
Image("/Users/macbook/Documents/Personnel/Car-Number-Plates-Detection/plates/s2222.png")

FileNotFoundError: No such file or directory: '/Users/macbook/Documents/Personnel/Car-Number-Plates-Detection/plates/s2222.png'

FileNotFoundError: No such file or directory: '/Users/macbook/Documents/Personnel/Car-Number-Plates-Detection/plates/s2222.png'

<IPython.core.display.Image object>

In [4]:

reader = easyocr.Reader(['en'])

In [5]:

output = reader.readtext('/Users/macbook/Documents/Personnel/Car-Number-Plates-Detection/plates/2222.png')

In [6]:
output

[([[np.int32(311), 0],
   [np.int32(389), 0],
   [np.int32(389), np.int32(17)],
   [np.int32(311), np.int32(17)]],
  'Lockea',
  np.float64(0.9128305953663051)),
 ([[np.int32(1003), np.int32(645)],
   [np.int32(1245), np.int32(645)],
   [np.int32(1245), np.int32(689)],
   [np.int32(1003), np.int32(689)]],
  'FeDG',
  np.float64(0.07322616875171661)),
 ([[np.int32(439), np.int32(747)],
   [np.int32(1013), np.int32(747)],
   [np.int32(1013), np.int32(855)],
   [np.int32(439), np.int32(855)]],
  'KA 64 N 0099',
  np.float64(0.8557685223331126))]

In [7]:
cord = output[-1][0]

In [8]:
cord

[[np.int32(439), np.int32(747)],
 [np.int32(1013), np.int32(747)],
 [np.int32(1013), np.int32(855)],
 [np.int32(439), np.int32(855)]]

In [9]:
a = list(zip(*cord))
a

[(np.int32(439), np.int32(1013), np.int32(1013), np.int32(439)),
 (np.int32(747), np.int32(747), np.int32(855), np.int32(855))]

In [10]:
min(a[0])

np.int32(439)

In [11]:
min(a[1])

np.int32(747)

In [12]:
max(a[0])

np.int32(1013)

In [13]:
max(a[1])

np.int32(855)

In [14]:

x_min, y_min = [int(min(idx)) for idx in zip(*cord)]

In [15]:
x_min, y_min

(439, 747)

In [16]:

x_max, y_max = [int(max(idx)) for idx in zip(*cord)]

In [17]:
x_max, y_max

(1013, 855)

In [18]:
from pylab import rcParams
rcParams['figure.figsize'] = 20, 30

In [19]:
image = cv2.imread('/content/bottle.jpg')
cv2.rectangle(image,(x_min,y_min),(x_max,y_max),(0,0,255),2)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

[ WARN:0@11.581] global loadsave.cpp:268 findDecoder imread_('/content/bottle.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
